In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [ ]:
# eda on data -> build models -> isolation forest , Local Outlier Factor (LOF) , xgboost , lightgbm -> mlflow, dvc -> streamlit app

In [ ]:
df = pd.read_csv('../paySim/PaySim_log.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df['isFraud'].value_counts()

In [ ]:
sns.countplot(x='isFraud', data=df, hue='isFraud')
plt.show()

In [ ]:
df['isFlaggedFraud'].value_counts()

In [ ]:
print(df['type'].value_counts())
transaction_types = df['type'].value_counts()

In [ ]:
plt.pie(transaction_types.values, labels=transaction_types.index, autopct='%1.1f%%') # type: ignore
plt.title('Transaction Type Distribution')
plt.show()


In [ ]:
df.groupby('type')['isFraud'].mean().sort_values(ascending=False)

In [ ]:
fraud_by_type = df.groupby('type')['isFraud'].mean().sort_values(ascending=False)
plt.figure(figsize=(10,6))
sns.barplot(x=fraud_by_type.index, y=fraud_by_type.values, hue=fraud_by_type.index,palette='Set1')
plt.title('Fraud Rate by Transaction Type')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
df['hour'] = df['step'] % 24
df['day'] = (df['step'] // 24) % 7
df['is_weekend'] = (df['day'] >= 5).astype(int)
df.head()

In [ ]:
fraud_by_hour = df.groupby('hour')['isFraud'].mean()
plt.plot(fraud_by_hour.index, fraud_by_hour.values, marker='o') # type: ignore
plt.xlabel('Hour of Day')
plt.ylabel('Fraud Rate')
plt.title('Fraud Rate by Hour')
plt.show()

In [ ]:
volume_by_hour = df.groupby('hour').size()
plt.plot(volume_by_hour.index, volume_by_hour.values, marker='o', color='orange')
plt.xlabel('Hour of Day')
plt.ylabel('Transaction Volume')
plt.title('Transaction Volume by Hour')
plt.show()

In [ ]:
fraud_by_day = df.groupby('day')['isFraud'].mean()
sns.barplot(x = fraud_by_day.index, y = fraud_by_day.values, hue=fraud_by_day.index, palette = 'Set1') # type: ignore
plt.xlabel('Day of Week')
plt.ylabel('Fraud Rate')
plt.title('Fraud Rate by Day of Week')
plt.show()

In [ ]:
fraud_weekend = df.groupby('is_weekend')['isFraud'].mean()
sns.barplot(x = ['Weekday', 'Weekend'], y = fraud_weekend.values, hue=['Weekday', 'Weekend'], palette = 'Set2') # type: ignore
plt.ylabel('Fraud Rate')
plt.title('Fraud Rate: Weekend vs Weekday')
plt.show()

In [ ]:
amount_by_step = df.groupby('step')['amount'].mean()
plt.plot(amount_by_step.index, amount_by_step.values, alpha=0.7) # type: ignore
plt.xlabel('Time Step')
plt.ylabel('Average Transaction Amount')
plt.title('Transaction Amount Trend Over Time')
plt.show()

In [ ]:
fraud_transactions = df[df['isFraud'] == 1]
fraud_by_step = fraud_transactions.groupby('step').size()
plt.plot(fraud_by_step.index, fraud_by_step.values, color='red', alpha=0.7) # type: ignore
plt.xlabel('Time Step')
plt.ylabel('Number of Fraud Transactions')
plt.title('Fraud Transactions Over Time')
plt.show()

In [ ]:
plt.hist(df['amount'], bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('Transaction Amount')
plt.ylabel('Frequency')
plt.title('Distribution of Transaction Amounts')
plt.yscale('log')
plt.show()

In [ ]:
for fraud_status in [0, 1]:
    subset = df[df['isFraud'] == fraud_status]
    plt.hist(subset['amount'], bins=50, alpha=0.6, 
             label=f'Fraud: {fraud_status}', log=True)
plt.xlabel('Transaction Amount')
plt.ylabel('Frequency (log scale)')
plt.title('Amount Distribution by Fraud Status')
plt.legend()

In [ ]:
sns.boxplot(x='isFraud', y='amount', data=df)
plt.yscale('log')
plt.title('Transaction Amount by Fraud Status')
plt.show()

In [ ]:
top_transactions = df.nlargest(100, 'amount')
sns.scatterplot(x='step', y='amount', hue='isFraud', data=top_transactions)
plt.title('Top 100 Largest Transactions')
plt.ylabel('Amount')
plt.show()

In [ ]:
fraud_stats = df.groupby('isFraud')['amount'].agg(['mean', 'median', 'std', 'min', 'max'])
fraud_stats.plot(kind='bar', subplots=True, layout=(2, 3), figsize=(15, 10))
plt.tight_layout()


In [ ]:
for fraud_status in [0, 1]:
    subset = df[df['isFraud'] == fraud_status]['amount']
    plt.hist(subset, bins=1000, cumulative=True, density=True, 
             histtype='step', alpha=0.8, label=f'Fraud: {fraud_status}')
plt.xscale('log')
plt.xlabel('Transaction Amount (log scale)')
plt.ylabel('CDF')
plt.title('Cumulative Distribution of Amounts')
plt.legend()

In [ ]:
# Balance features analysis
balance_features = ['oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']

plt.figure(figsize=(15, 10))

for i, feature in enumerate(balance_features, 1):
    plt.subplot(2, 2, i)
    plt.hist(np.log1p(df[feature]), bins=50, alpha=0.7, edgecolor='black')
    plt.xlabel(f'Log({feature} + 1)')
    plt.ylabel('Frequency')
    plt.title(f'Distribution of {feature}')

plt.tight_layout()
plt.show()

In [ ]:
df['balance_change_org'] = df['newbalanceOrig'] - df['oldbalanceOrg']
df['balance_change_dest'] = df['newbalanceDest'] - df['oldbalanceDest']

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.boxplot(x='isFraud', y='balance_change_org', data=df)
plt.title('Originator Balance Change by Fraud Status')

plt.subplot(1, 2, 2)
sns.boxplot(x='isFraud', y='balance_change_dest', data=df)
plt.title('Destination Balance Change by Fraud Status')

plt.tight_layout()
plt.show()

In [ ]:
df.head()

In [ ]:
customer_orig_stats = df.groupby('nameOrig').agg({
    'amount': ['count', 'mean', 'std', 'sum'],
    'isFraud': 'mean'
}).round(2)

customer_dest_stats = df.groupby('nameDest').agg({
    'amount': ['count', 'mean', 'std', 'sum'],
    'isFraud': 'mean'
}).round(2)

In [ ]:
print("Top 10 Originators by Transaction Count:")
print(customer_orig_stats.nlargest(10, ('amount', 'count'))) # type: ignore

print("\nTop 10 Destinations by Transaction Count:")
print(customer_dest_stats.nlargest(10, ('amount', 'count'))) # type: ignore

In [ ]:
transaction_counts_orig = df['nameOrig'].value_counts()
plt.hist(transaction_counts_orig, bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('Transactions per Originator')
plt.ylabel('Frequency')
plt.title('Distribution of Transactions per Originator')
plt.yscale('log')
plt.show()

In [ ]:
transaction_counts_dest = df['nameDest'].value_counts()
plt.hist(transaction_counts_dest, bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('Transactions per Destination')
plt.ylabel('Frequency')
plt.title('Distribution of Transactions per Destination')
plt.yscale('log')
plt.show()

In [ ]:
df['orig_txn_count'] = df.groupby('nameOrig')['nameOrig'].transform('count')
activity_bins = pd.cut(df['orig_txn_count'], bins=[0, 1, 5, 10, 50, 100, float('inf')])
fraud_by_activity = df.groupby(activity_bins)['isFraud'].mean()
fraud_by_activity.plot(kind='bar')
plt.xlabel('Transaction Count Bins')
plt.ylabel('Fraud Rate')
plt.title('Fraud Rate by Customer Activity Level')
plt.xticks(rotation=45)
plt.show()

In [ ]:
numeric_features = ['amount', 'oldbalanceOrg', 'newbalanceOrig', 
                   'oldbalanceDest', 'newbalanceDest', 'balance_change_org', 
                   'balance_change_dest']
correlation_matrix = df[numeric_features + ['isFraud']].corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, fmt='.2f')
plt.title('Correlation Matrix')
plt.tight_layout()
plt.show()

In [ ]:
fraud_sample = df[df['isFraud'] == 1].sample(min(1000, len(df[df['isFraud'] == 1])))
non_fraud_sample = df[df['isFraud'] == 0].sample(min(1000, len(fraud_sample)))

sample_df = pd.concat([fraud_sample, non_fraud_sample])

plt.figure(figsize=(12, 10))
sns.pairplot(sample_df, vars=numeric_features[:4], hue='isFraud', 
             diag_kind='hist', palette='viridis', plot_kws={'alpha': 0.6})
plt.suptitle('Pairplot of Numerical Features (Sampled)', y=1.02)
plt.show()

In [ ]:
df['amount_to_balance_ratio'] = df['amount'] / (df['oldbalanceOrg'] + 1)
df['is_zero_balance_orig'] = (df['oldbalanceOrg'] == 0).astype(int)
df['is_zero_balance_dest'] = (df['oldbalanceDest'] == 0).astype(int)

patterns_to_analyze = [
    'amount_to_balance_ratio',
    'is_zero_balance_orig',
    'is_zero_balance_dest'
]
plt.figure(figsize=(15, 5))

for i, pattern in enumerate(patterns_to_analyze, 1):
    plt.subplot(1, 3, i)
    
    if 'ratio' in pattern:
        # Log scale for ratios
        for fraud_status in [0, 1]:
            subset = df[df['isFraud'] == fraud_status][pattern]
            plt.hist(np.log1p(subset), bins=50, alpha=0.6, 
                     label=f'Fraud: {fraud_status}', density=True)
        plt.xlabel(f'Log({pattern} + 1)')
    else:
        fraud_rates = df.groupby(pattern)['isFraud'].mean()
        plt.bar(fraud_rates.index, fraud_rates.values)
        plt.xlabel(pattern)
    
    plt.ylabel('Density' if 'ratio' in pattern else 'Fraud Rate')
    plt.title(f'Fraud Analysis: {pattern}')
    if 'ratio' in pattern:
        plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
print("Comparison of Flagged Fraud vs Actual Fraud:")
confusion_matrix = pd.crosstab(df['isFlaggedFraud'], df['isFraud'])
print(confusion_matrix)

precision = confusion_matrix.loc[1, 1] / confusion_matrix.loc[1].sum() if 1 in confusion_matrix.index else 0 # type: ignore
recall = confusion_matrix.loc[1, 1] / confusion_matrix.loc[:, 1].sum() if 1 in confusion_matrix.index else 0
print(f"\nFlagged Fraud Precision: {precision:.2%}")
print(f"Flagged Fraud Recall: {recall:.2%}")

In [ ]:
from scipy.stats import mannwhitneyu, chi2_contingency

# Statistical tests for significant differences
print("=== STATISTICAL SIGNIFICANCE TESTS ===")

# Test 1: Amount difference between fraud and non-fraud
fraud_amounts = df[df['isFraud'] == 1]['amount']
non_fraud_amounts = df[df['isFraud'] == 0]['amount']
stat, p_value = mannwhitneyu(fraud_amounts, non_fraud_amounts)
print(f"Amount difference (Mann-Whitney U): p-value = {p_value:.2e}")

# Test 2: Transaction type and fraud association
contingency_table = pd.crosstab(df['type'], df['isFraud'])
chi2, p_value, dof, expected = chi2_contingency(contingency_table)
print(f"Transaction type association (Chi-square): p-value = {p_value:.2e}")

# Key business insights summary
print("\n=== KEY BUSINESS INSIGHTS ===")
print(f"1. Overall fraud rate: {df['isFraud'].mean():.2%}")
print(f"2. Most common fraud type: {fraud_by_type.index[0]} ({fraud_by_type.iloc[0]:.2%} fraud rate)")
print(f"3. Peak fraud hour: {fraud_by_hour.idxmax()} ({fraud_by_hour.max():.2%} fraud rate)")
print(f"4. Average fraud transaction amount: ${fraud_amounts.median():.2f}")
print(f"5. Fraud detection system recall: {recall:.2%}")

# Risk factors identification
print("\n=== IDENTIFIED RISK FACTORS ===")
risk_factors = []
if fraud_by_type.max() > 0.1:
    risk_factors.append(f"High fraud concentration in {fraud_by_type.index[0]} transactions")
if fraud_by_hour.max() > fraud_by_hour.median() * 2:
    risk_factors.append(f"Temporal clustering of fraud during hour {fraud_by_hour.idxmax()}")
if df['amount_to_balance_ratio'].max() > 10:
    risk_factors.append("Large transactions relative to account balance")

for i, factor in enumerate(risk_factors, 1):
    print(f"{i}. {factor}")

In [ ]:
df.head()

In [ ]:
import os
os.makedirs('processeddataset', exist_ok=True)

In [ ]:
df.to_csv('processeddataset/processed_paySim.csv', index=False)